In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, embed_properties_into_html
from method.ours.preprocessing import get_processable_nodes
from method.ours.graph_embedding import create_node2vec_model
from method.ours.relation_graph import create_relation_graph
from method.ours.node_linking import create_base_links
from method.ours.similarity import add_similarity_scores_to_graph
from method.ours.edge_pruning import (
    prune_relation_graph_extra_edges,
    prune_low_score_uncertain_edges
)
from method.ours.input_group import (
    create_input_groups,
    prune_low_score_group_relations
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

In [4]:
driver = create_driver(HEADLESS)
driver.get('https://www.aircanada.com/ca/en/aco/home.html')

[WDM] - Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.29M/8.29M [00:00<00:00, 11.3MB/s]


In [5]:
# form = driver.find_elements(By.TAG_NAME, 'form')[48]
# form = driver.find_element(By.ID, 'register')
form = driver.find_elements(By.TAG_NAME, 'form')[1]

# Processing

In [6]:
# def parse_form(form):
form = embed_properties_into_html(driver, form)
form_doc = bs(form.get_attribute('outerHTML'), 'html.parser')

form_processable_nodes = get_processable_nodes(form_doc)
node2vec_model = create_node2vec_model(form_doc)

relation_graph = create_relation_graph(form_processable_nodes, TEXT_EMBEDDING_METHOD)

relation_graph = create_base_links(relation_graph)

relation_graph = add_similarity_scores_to_graph(node2vec_model, relation_graph)

relation_graph = prune_relation_graph_extra_edges(relation_graph)
relation_graph = prune_low_score_uncertain_edges(relation_graph, factor=0.5)

input_groups = create_input_groups(relation_graph)
input_groups = prune_low_score_group_relations(input_groups, node2vec_model, factor=0.5)
# return input_groups

Computing transition probabilities:   0%|          | 0/340 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:08<00:00,  4.09it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [7]:
# t0 = time.time()
# input_groups = parse_form(form)
# print('running time:', time.time() - t0)

# Generation

In [19]:
from method.ours.constraints import (
    ConstraintFactory,
    create_constraint_generation_prompt,
    generate_constraints_with_llm
)
from method.ours.values import (
    create_value_generation_prompt,
    generate_values_with_llm,
)

In [9]:
input_group = input_groups[8]
input_group.node.element

<input aria-describedby="bkmgFlights_travelDates_1DatepickerSrHelperText" aria-disabled="false" aria-invalid="false" aria-label="Return" autocomplete="off" autocorrect="off" class="abc-form-element-input abc-form-element-main text-transform-none ng-untouched ng-pristine ng-valid ng-star-inserted" id="bkmgFlights_travelDates_1-formfield-2" name="bkmgFlights_travelDates_1-formfield-2" placeholder="DD/MM" spellcheck="false" type="text" x_end="1093" x_start="978" xpath="//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-DATE-PICKER[1]/DIV[1]/DIV[1]/ABC-INPUT[2]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1]" y_end="678" y_start="628"/>

In [13]:
constraint_user_prompt = create_constraint_generation_prompt(input_group)
print(constraint_user_prompt)

input: <input id="bkmgFlights_travelDates_1-formfield-2" name="bkmgFlights_travelDates_1-formfield-2" type="text"/>
with label: Return
with the following relevant text tags:
<span>Return</span>

    The relevant information available in the form are (in order of relevance):
    1.
input: <input id="bkmgFlights_travelDates_1-formfield-1" name="bkmgFlights_travelDates_1-formfield-1" type="text"/>
with label: Departure
2.
input: <input id="bkmgFlights_destination_trip_1" name="bkmgFlights_destination_trip_1" type="text"/>
with label: To
3.
input: <input id="bkmgFlights_tripTypeSelector_M" name="tripType" type="radio"/>
with label: Multi-city/Stopover


In [14]:
generated_constraints = generate_constraints_with_llm(
    constraint_user_prompt,
    openai_api_key=openai.api_key
)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of the constraints and their signatures is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondition(condition, value) # the input field value has the given condition to the given value\n5. toContainSubstr(value) # the input field value contains a specific string\n6. toContainChar(value) # the input field value contains a specific character\n7. toBeAlpha() # the input field value is alphabetic\n8. toBeNumeric() # the input field value is numeric\n9. toBeAlphaNumeric() # the input value con

In [17]:
constraints = list(map(
    ConstraintFactory.create,
    generated_constraints.split('\n.')[1:]
))

In [21]:
value_user_prompt = create_value_generation_prompt(
    input_group,
    constraints
)
print(value_user_prompt)

input: <input id="bkmgFlights_travelDates_1-formfield-2" name="bkmgFlights_travelDates_1-formfield-2" type="text"/>
with label: Return
with the following relevant text tags:
<span>Return</span>

    The relevant information available in the form are (in order of relevance):
    1.
input: <input id="bkmgFlights_travelDates_1-formfield-1" name="bkmgFlights_travelDates_1-formfield-1" type="text"/>
with label: Departure
2.
input: <input id="bkmgFlights_destination_trip_1" name="bkmgFlights_destination_trip_1" type="text"/>
with label: To
3.
input: <input id="bkmgFlights_tripTypeSelector_M" name="tripType" type="radio"/>
with label: Multi-city/Stopover

    generate values based on the following constraints:
    input field should be non-empty
input field should be  field('bkmgFlights_travelDates_1-formfield-1')


In [23]:
generated_values = generate_values_with_llm(
    value_user_prompt,
    openai_api_key=openai.api_key
)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate a set of values for a form field based on the form field information and a set of constraints on the field. Your decisions must always be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate five distinct values that satisfy the constraints while keeping in mind the nature of the input from the available information.\nOnly generate the values and don't explain your answers.\nGenerate the values in a Python array. We must be able to parse your generation with json.loads.\nOnly generate values for the inputs in question, and not the ones in the relevant information section.\n"
  },
  {
    "role": "user",
    "content": "We are generating values for the following input field:\n            input: <input id=\"bkmgFlights_travelDates_1-formfield-2\" name=\"bkmgFlights_travelDates_1-formfield-2\" type=\"text\"/>\nwith label: Return\nwith the fo

In [24]:
print(generated_values)

["2022-10-15", "2022-10-20", "2022-11-01", "2022-11-10", "2022-12-05"]
